In [1]:
from dotenv import load_dotenv
import os

# loads variables from .env
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("Environment variable GOOGLE_API_KEY not found!")

print("Secret loaded successfully!")

Secret loaded successfully!


In [5]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Gemini API key setup complete.")

✅ Gemini API key setup complete.


In [7]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [9]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

---
## ➰ Section 5: Loop Workflows - The Refinement Cycle

**The Problem: One-Shot Quality**

All the workflows we've seen so far run from start to finish. The `SequentialAgent` and `ParallelAgent` produce their final output and then stop. This 'one-shot' approach isn't good for tasks that require refinement and quality control. What if the first draft of our story is bad? We have no way to review it and ask for a rewrite.

**The Solution: Iterative Refinement**

When a task needs to be improved through cycles of feedback and revision, you can use a `LoopAgent`. A `LoopAgent` runs a set of sub-agents repeatedly *until a specific condition is met or a maximum number of iterations is reached.* This creates a refinement cycle, allowing the agent system to improve its own work over and over.

**Use Loop when:** Iterative improvement is needed, quality refinement matters, or you need repeated cycles.

To learn more, check out the documentation related to [loop agents in ADK](https://google.github.io/adk-docs/agents/workflow-agents/loop-agents/).

**Architecture: Story Writing & Critique Loop**

```mermaid
graph TD
    A["Initial Prompt"] -- > B["Writer Agent"]
    B -- >|story| C["Critic Agent"]
    C -- >|critique| D{"Iteration < Max<br>AND<br>Not Approved?"}
    D -- >|Yes| B
    D -- >|No| E["Final Story"]

    style B fill:#ccffcc
    style C fill:#ffcccc
    style D fill:#ffffcc
```

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

### 5.1 Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [18]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [20]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The `LoopAgent` itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

1. A simple Python function that the `LoopAgent` understands as an "exit" signal.
2. An agent that can call that function when the right condition is met.

First, you'll define the `exit_loop` function:

In [23]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


To let an agent call this Python function, we wrap it in a `FunctionTool`. Then, we create a `RefinerAgent` that has this tool.

👉 **Notice its instructions:** this agent is the "brain" of the loop. It reads the `{critique}` from the `CriticAgent` and decides whether to (1) call the `exit_loop` tool or (2) rewrite the story.

In [26]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of `max_iterations`:

In [29]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [31]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > The salt spray stung Elias’s cheeks as he ascended the winding stairs, the rhythmic sweep of the lamp overhead a familiar lullaby. For forty years, the lighthouse had been his world, its beam a solitary sentinel against the crushing dark. Tonight, however, a different light pulsed from the small, oak desk. Tucked beneath a stack of tide charts was a rolled parchment. As he unfurled it, a soft, cerulean glow emanated, illuminating intricate lines that weren’t of coastlines he knew. Islands shimmered with an inner light, and symbols, etched in a script as ancient as the sea itself, pulsed with an ethereal energy. It was a map, unlike any he’d ever seen, promising wonders beyond the horizon.
CriticAgent > The plot is intriguing, setting up a classic adventure premise. However, the story feels like an introduction and lacks a clear n

In [39]:
from IPython.display import display, Image, Markdown

text = ""
for _response in response:
    if _response is not None and _response.content is not None and _response.content.parts is not None:
        for part in _response.content.parts:
            if part.text is not None:
                text += part.text

Markdown(text)

The salt spray stung Elias’s cheeks as he ascended the winding stairs, the rhythmic sweep of the lamp overhead a familiar lullaby. For forty years, the lighthouse had been his world, its beam a solitary sentinel against the crushing dark. Tonight, however, a different light pulsed from the small, oak desk. Tucked beneath a stack of tide charts was a rolled parchment. As he unfurled it, a soft, cerulean glow emanated, illuminating intricate lines that weren’t of coastlines he knew. Islands shimmered with an inner light, and symbols, etched in a script as ancient as the sea itself, pulsed with an ethereal energy. It was a map, unlike any he’d ever seen, promising wonders beyond the horizon.The plot is intriguing, setting up a classic adventure premise. However, the story feels like an introduction and lacks a clear narrative arc for Elias beyond the discovery itself.

Here are a few suggestions for improvement:

1.  **Develop Elias's reaction and motivation:** While Elias is described as having spent forty years in the lighthouse, his internal reaction to this extraordinary discovery is brief. What are his immediate thoughts, fears, or desires upon seeing the map? Does his lifelong routine suddenly feel confining, or is he skeptical, or is he filled with a sudden yearning for adventure? Expanding on this will make him a more compelling protagonist.
2.  **Hint at the map's purpose or origin:** The map is described as ancient and ethereal. Adding a subtle clue about *why* it's in the lighthouse, or who might have left it, could deepen the mystery and provide Elias with a stronger motivation to act. Is it a legacy, a trap, or a calling?
3.  **Introduce a tangible next step or conflict:** The story ends with the promise of wonders. To make it feel more complete as a short story, consider adding a small, immediate action Elias takes or a decision he makes as a direct result of finding the map. This could be something as simple as deciding to research the symbols, or perhaps a storm begins to brew, mirroring his internal turmoil and forcing a choice.The salt spray stung Elias’s cheeks as he ascended the winding stairs, the rhythmic sweep of the lamp overhead a familiar lullaby. For forty years, the lighthouse had been his world, its beam a solitary sentinel against the crushing dark. Tonight, however, a different light pulsed from the small, oak desk. Tucked beneath a stack of tide charts, a rolled parchment felt oddly heavy. As he unfurled it, a soft, cerulean glow emanated, illuminating intricate lines that weren’t of coastlines he knew. Islands shimmered with an inner light, and symbols, etched in a script as ancient as the sea itself, pulsed with an ethereal energy. It was a map, unlike any he’d ever seen.

A tremor ran through Elias’s weathered hands, a feeling entirely alien to the steady routine of his life. Forty years. Forty years of marking the passage of time by the lamp’s sweep and the ocean’s roar. This map… it felt like a ghost whispering from the past, a forgotten legacy left by the lighthouse’s previous keeper, Old Man Hemlock, a man rumored to have sailed beyond the known world. The map hummed with a silent invitation, a stark contrast to the predictable solitude he’d always accepted. A yearning, sharp and unexpected, pierced through his ingrained skepticism. Could this be a calling?

The glowing symbols seemed to shift, coalescing into what looked like a stylized kraken, then reforming into a constellation he’d never observed. One particular island pulsed brighter than the rest, labeled with a single, looping script that Elias, with his lifelong study of maritime lore, recognized as a variation of a long-lost Phoenician dialect, meaning ‘haven’. A haven? Or a trap? As if in response to his turmoil, the wind outside began to howl, a sudden squall lashing against the glass. The sea, his constant companion, seemed to be roaring its own question. Elias looked from the map to the tempest outside, a decision solidifying in his chest. He carefully rolled the parchment, its glow now a comforting warmth against his palm. Tomorrow, he would consult the dusty, forgotten tomes in the lighthouse’s neglected library. He would decipher these symbols. The horizon had always been a boundary; tonight, it had become a promise.APPROVED